In [ ]:
%pip install wikiframe 

In [1]:
from wikiframe import Extractor
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from utils import get_int_col

In [8]:
extractor = Extractor('data')
dict_df = extractor.extract_from_csv()

In [432]:
train = dict_df['house_train_raw']
test = dict_df['houses_test_raw']

In [433]:
test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [427]:
train.shape

(1460, 81)

In [107]:
#get int values

def get_int_col(df):
    int_col = []
    for col in df.keys():
        if df.dtypes[col] == np.dtype(np.int64):
            int_col.append(col)
    return int_col

In [316]:
def get_num_col(df):
    object_col = []
    for col in df.keys():
        if df.dtypes[col] != np.dtype(object):
            object_col.append('MSSubClass')
            object_col.append(col)
    return object_col

In [228]:
#get object values
def get_object_col(df):
    object_col = []
    for col in df.keys():
        if df.dtypes[col] == np.dtype(object):
            object_col.append('MSSubClass')
            object_col.append(col)
    return object_col

len(get_object_col(train))

86

In [197]:
def onehot(df,list):
    onehot_dict = {}
    encoder = OneHotEncoder(handle_unknown='ignore')
    for name in df.keys():
        if df.dtypes[name] == np.dtype(object):
            onehot_dict[name] = pd.DataFrame(encoder.fit_transform(df[[name]]),columns=df[name].unique())
        onehot_dict['MSSubClass'] = pd.DataFrame(encoder.fit_transform(df[['MSSubClass']]),columns=df['MSSubClass'].unique())
    return onehot_dict

In [140]:
train['MSZoning'].unique()

array(['RL', 'RM', 'C (all)', 'FV', 'RH'], dtype=object)

In [143]:
test['MSZoning'].unique()

array(['RH', 'RL', 'RM', 'FV', 'C (all)', nan], dtype=object)

In [134]:
onehot = onehot(train)

In [135]:
onehot['MSZoning']

,RL,RM,C (all),FV,RH
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...
1455,0.0,0.0,0.0,1.0,0.0
1456,0.0,0.0,0.0,1.0,0.0
1457,0.0,0.0,0.0,1.0,0.0
1458,0.0,0.0,0.0,1.0,0.0


In [126]:
onehot['Street']

,Grvl,Pave
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
1455,0,1
1456,0,1
1457,0,1
1458,0,1


In [189]:
def get_unuque(df):
    val = {}
    for col in df.keys():
        val[col] = df[col].unique()
    return val

In [190]:
set(get_unuque(train)['MSSubClass']) == set(get_unuque(test)['MSSubClass'])

False

In [147]:
def get_unque(df):
    val = {}
    for col in df.keys():
        if df.dtypes[col] == np.dtype(object):
            val[col] = df[col].unique()
    return val

In [192]:
set(get_unque(train)['Street']) == set(get_unque(test)['Street'])

True

In [186]:
def compare_df(df1,df2):
    com = []
    for name in df1:
        if df1.dtypes[name] == np.dtype(object):
            if (set(get_unque(df1)[name]) == set(get_unque(df2)[name])) == False:
                com.append(name)
    return com

In [206]:
pd.get_dummies(train['RoofMatl'].astype('category').cat.codes.unique())

,0,1,2,3,4,5,6,7
0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0
5,0,0,0,0,0,1,0,0
6,0,0,0,0,1,0,0,0
7,1,0,0,0,0,0,0,0


In [210]:
test.shape

(1459, 80)

In [262]:
df_all = pd.concat([train,test])

In [391]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[feature_to_encode])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

In [407]:
df_all = pd.concat([train,test])

In [408]:
df_all['MSSubClass']

0        60
1        20
2        60
3        70
4        60
       ... 
1454    160
1455    160
1456     20
1457     85
1458     60
Name: MSSubClass, Length: 2919, dtype: int64

In [425]:
def one_hot_transform(df,features):
    for feature in features:
        if df.dtypes[feature] == np.dtype(object):
            df =  encode_and_bind(df,feature)
    return df

In [426]:
df_all = pd.concat([train,test])

with open("features.txt", "r") as f:
    features = " ".join([l.rstrip("\n") for l in f]) 
    features = features.split()

df_all = df_all[features]
    
df_all = one_hot_transform(df_all,features)

print(df_all.shape)
df_all.keys().values

(2919, 83)


array(['LotArea', 'OverallCond', 'YearBuilt', 'FullBath', 'TotRmsAbvGrd',
       'OpenPorchSF', 'PoolArea', 'SalePrice', 'C (all)', 'FV', 'RH',
       'RL', 'RM', 'IR1', 'IR2', 'IR3', 'Reg', 'AllPub', 'NoSeWa',
       'Corner', 'CulDSac', 'FR2', 'FR3', 'Inside', 'Blmngtn', 'Blueste',
       'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards',
       'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NPkVill',
       'NWAmes', 'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer',
       'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker', '1Fam',
       '2fmCon', 'Duplex', 'Twnhs', 'TwnhsE', '1.5Fin', '1.5Unf',
       '1Story', '2.5Fin', '2.5Unf', '2Story', 'SFoyer', 'SLvl', 'Ex',
       'Fa', 'Gd', 'TA', 'Ex', 'Fa', 'Gd', 'Po', 'TA', 'N', 'P', 'Y',
       'COD', 'CWD', 'Con', 'ConLD', 'ConLI', 'ConLw', 'New', 'Oth', 'WD'],
      dtype=object)

In [428]:
train = df_all.iloc[:1460,:]

In [434]:
test = df_all.iloc[1460:,:]
test

,LotArea,OverallCond,YearBuilt,FullBath,TotRmsAbvGrd,OpenPorchSF,PoolArea,SalePrice,C (all),FV,...,Y,COD,CWD,Con,ConLD,ConLI,ConLw,New,Oth,WD
0,11622,6,1961,1,5,0,0,NaN,0,0,...,1,0,0,0,0,0,0,0,0,1
1,14267,6,1958,1,6,36,0,NaN,0,0,...,1,0,0,0,0,0,0,0,0,1
2,13830,5,1997,2,6,34,0,NaN,0,0,...,1,0,0,0,0,0,0,0,0,1
3,9978,6,1998,2,7,36,0,NaN,0,0,...,1,0,0,0,0,0,0,0,0,1
4,5005,5,1992,2,5,82,0,NaN,0,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1936,7,1970,1,5,0,0,NaN,0,0,...,1,0,0,0,0,0,0,0,0,1
1455,1894,5,1970,1,6,24,0,NaN,0,0,...,1,0,0,0,0,0,0,0,0,1
1456,20000,7,1960,1,7,0,0,NaN,0,0,...,1,0,0,0,0,0,0,0,0,1
1457,10441,5,1992,1,6,32,0,NaN,0,0,...,1,0,0,0,0,0,0,0,0,1
